In [ ]:
pip install mysql-connector-python

In [ ]:
import pandas as pd
import numpy as np
import importlib.metadata
import mysql.connector
import streamlit as st

from datetime import datetime

In [48]:
# Cargar csv
df_coches = pd.read_csv("../bin/coche_prueba.csv")
df_prueba = pd.read_csv("../bin/prueba.csv")
df_carrocerias = pd.read_csv("../bin/listado_carrocerias.csv")
df_combustibles = pd.read_csv("../bin/listado_combustibles.csv")
df_concesionarios = pd.read_csv("../bin/listado_concesionarios.csv")
df_distintivo_ambiental = pd.read_csv("../bin/listado_distintivo_ambiental.csv")
df_marcas = pd.read_csv("../bin/listado_marcas.csv")
df_modelos = pd.read_csv("../bin/listado_modelos.csv")
df_provincias = pd.read_csv("../bin/listado_provincias.csv")
df_sobrealimentacion = pd.read_csv("../bin/listado_sobrealimentacion.csv")
df_tipos_cambio = pd.read_csv("../bin/listado_tipos_cambio.csv")
df_prestaciones = pd.read_csv("../bin/prestaciones.csv")
df_modelos

,id_marca,nombre_marca
0,1,PORSCHE
1,2,DACIA
2,3,VOLKSWAGEN
3,4,CITROEN
4,5,SKODA
5,6,PEUGEOT
6,7,FORD
7,8,LAND-ROVER
8,9,BMW
9,10,MERCEDES-BENZ


In [ ]:
# Conectar a la base de datos usando los valores almacenados en `secrets.toml`
def conectar_base_datos():
    conn = mysql.connector.connect(
        host=st.secrets["database"]["host"],
        user=st.secrets["database"]["user"],
        password=st.secrets["database"]["password"],
        database=st.secrets["database"]["database"]
    )
    return conn

In [46]:
# Funcion para insertar coches en la base de datos
def insertar_datos(df, nombre_tabla):
    # Conexión a la base df_cochedatos de MySQL
    conn = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = "Whereisbotus1996",
        database = "coches_segunda_mano"
    )
    # Crear un cursor para ejecutar comandos SQL
    cursor = conn.cursor()
    # Obtener las columnas del DataFrame
    columnas = df.columns.tolist()
    # Convertir cada fila del DataFrame a una tupla
    valores = [tuple(row) for row in df.to_numpy()]

    # Crear la consulta SQL para insertar datos
    query = f"""
        INSERT INTO {nombre_tabla} ({', '.join(columnas)})
        VALUES ({', '.join(['%s'] * len(columnas))})
    """

    # Ejecutar la consulta para todas las filas
    cursor.executemany(query, valores)

    # Confirmar los cambios en la base de datos
    conn.commit()

    # Cerrar el cursor y la conexion
    cursor.close()
    conn.close()

    # Mensaje de exito
    print("Datos insertados correctamente")

# Llamar a la funcion para insertar los datos en la tabla
insertar_datos(df_provincias, 'provincias')

DatabaseError: 1364 (HY000): Field 'comunidad_autonoma' doesn't have a default value

In [ ]:
# Funcion para extraer datos
def extraer_datos(nombre_tabla):
    # Conexión a la base de datos de MySQL
    conn = conectar_base_datos()
    # Crear un cursor para ejecutar comandos SQL
    cursor = conn.cursor()

    # Crear la consulta SQL para seleccionar todos los datos de la tabla
    query = f"SELECT * FROM {nombre_tabla}"

    # Ejecutar la consulta y recuperar los datos
    cursor.execute(query)
    resultado = cursor.fetchall()

    # Obtener los nombres de las columnas de la tabla
    columnas = [desc[0] for desc in cursor.description]

    # Convertir los resultados a un DataFrame
    df = pd.DataFrame(resultado, columns=columnas)

    # Cerrar el cursor y la conexión
    cursor.close()
    conn.close()

    # Devolver el DataFrame
    return df

# Llamar a la función para extraer datos de la tabla
df_extraido = extraer_datos('marcas')
print(df_extraido)